In [1]:
#@title ⚙️ 1. 공통 환경 설정
import os
from IPython.display import display, Markdown

# 변수 정의 (이후 모든 셀에서 공통으로 사용됨)
카테고리 = "swoon" #@param {type:"string"}
base_path = f"/content/drive/MyDrive/project/AEGIS/Dataset/{카테고리}/image"

# 드라이브 마운트
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ 구글 드라이브 연결 완료")
else:
    print("✅ 드라이브가 이미 연결되어 있습니다.")

✅ 드라이브가 이미 연결되어 있습니다.


In [2]:
"""
실신(Swoon)
"""
def run_verification():
    if not os.path.exists(base_path):
        print(f"❌ 경로를 찾을 수 없습니다: {base_path}")
        return

    exclude_list = ["전체", "ipynb_checkpoints", ".ipynb_checkpoints"]
    folders = sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f not in exclude_list])

    problem_reports = [] # 문제가 있는 폴더만 저장
    error_count = 0
    fixed_count = 0
    total_folders_count = len(folders)

    for f_name in folders:
        f_path = os.path.join(base_path, f_name)
        status_list, is_all_8 = [], True

        for sub in ["이상", "의심", "정상"]:
            sub_path = os.path.join(f_path, sub)
            if os.path.exists(sub_path):

                # --- [파일명 정화 로직 강화] ---
                current_files = os.listdir(sub_path)
                for file in current_files:
                    # '의 사본', '복사본', '의 사본 (1)' 등 모든 케이스 대응 및 Jamo 형태의 '의 사본' 추가
                    targets_to_clean = ["의 사본", "사본", "복사본", "Copy", "의 사본"] # Jamo 형태 추가

                    if any(target in file for target in targets_to_clean):
                        # print(f"\n--- 파일명 정화 시도: {f_name}/{sub} ---") # 상세 로그 제거
                        # print(f"  원본 파일: {file}") # 상세 로그 제거
                        old_path = os.path.join(sub_path, file)

                        ext = ""
                        if ".jpg" in file.lower(): ext = ".jpg"
                        elif ".png" in file.lower(): ext = ".png"
                        elif ".jpeg" in file.lower(): ext = ".jpeg"

                        if ext:
                            # 확장자 앞부분만 추출 (예: xxxxx.jpg 까지만 남김)
                            clean_name = file.split(ext)[0] + ext
                            new_path = os.path.join(sub_path, clean_name)

                            # print(f"  예상 새 파일명: {clean_name}") # 상세 로그 제거
                            # print(f"  예상 새 경로: {new_path}") # 상세 로그 제거

                            try:
                                if not os.path.exists(new_path):
                                    os.rename(old_path, new_path)
                                    fixed_count += 1
                                    # print(f"  ✅ '{file}' -> '{clean_name}' (이름 변경 완료)") # 상세 로그 제거
                                else:
                                    # 만약 이미 깨끗한 파일이 있다면 사본은 그냥 삭제
                                    os.remove(old_path)
                                    fixed_count += 1
                                    # print(f"  🗑️ '{file}' (중복된 깨끗한 파일이 있어 삭제)") # 상세 로그 제거
                            except Exception as e:
                                print(f"  ⚠️ '{file}' 변경 중 오류: {e}") # 오류 로그는 유지
                        else:
                            print(f"  ❌ '{file}'은/는 지원되는 이미지 확장자를 찾을 수 없어 정화 대상에서 제외됩니다.") # 로그 유지
                # ------------------------------

                # 정화 후 파일 목록 다시 읽기
                final_files = [i for i in os.listdir(sub_path) if i.lower().endswith(('.jpg', '.jpeg', '.png'))]
                cnt = len(final_files)

                icon = "🟢" if cnt == 8 else "🔴"
                if cnt != 8: is_all_8 = False
                status_list.append(f"{icon} {sub}({cnt})")
            else:
                status_list.append(f"⚪ {sub}(미생성)")
                is_all_8 = False

        row_content = f"| {'✅' if is_all_8 else '❌'} | `{f_name}` | {' / '.join(status_list)} |"

        if not is_all_8:
            error_count += 1
            problem_reports.append(row_content) # 문제가 있는 폴더만 저장

    if fixed_count > 0:
        print(f"\n🛠  파일명 수정/정리 완료: 총 {fixed_count}건")

    normal_folders_count = total_folders_count - error_count

    # 전체 요약 보고서
    summary_markdown = f"""
### 📊 검증 보고서 (`{카테고리}`)
* 총 폴더 개수: {total_folders_count}개
* 정상 폴더 개수: {normal_folders_count}개
* 문제 발견 폴더: {error_count}개
"""
    display(Markdown(summary_markdown))

    if error_count == 0:
        display(Markdown("✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다."))
    else:
        # 문제 폴더를 모두 출력 (display_limit 제거)
        problem_report_markdown = f"""
---
#### ❌ 문제가 있는 폴더 상세 보고 (총 {error_count}개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
{       '\n' .join(problem_reports)}
"""
        display(Markdown(problem_report_markdown))

run_verification()


### 📊 검증 보고서 (`swoon`)
* 총 폴더 개수: 218개
* 정상 폴더 개수: 213개
* 문제 발견 폴더: 5개



---
#### ❌ 문제가 있는 폴더 상세 보고 (총 5개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
| ❌ | `100-4_cam01_swoon01_place02_day_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `106-6_cam01_swoon01_place05_day_spring` | 🔴 이상(14) / 🔴 의심(16) / 🔴 정상(16) |
| ❌ | `107-3_cam02_swoon01_place06_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `107-4_cam01_swoon01_place06_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `bgm` | ⚪ 이상(미생성) / ⚪ 의심(미생성) / ⚪ 정상(미생성) |


In [16]:
"""
파손(Vandalism)
"""
def run_verification():
    if not os.path.exists(base_path):
        print(f"❌ 경로를 찾을 수 없습니다: {base_path}")
        return

    exclude_list = ["전체", "ipynb_checkpoints", ".ipynb_checkpoints"]
    folders = sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f not in exclude_list])

    problem_reports = [] # 문제가 있는 폴더만 저장
    error_count = 0
    fixed_count = 0
    total_folders_count = len(folders)

    for f_name in folders:
        f_path = os.path.join(base_path, f_name)
        status_list, is_all_8 = [], True

        for sub in ["이상", "의심", "정상"]:
            sub_path = os.path.join(f_path, sub)
            if os.path.exists(sub_path):

                # --- [파일명 정화 로직 강화] ---
                current_files = os.listdir(sub_path)
                for file in current_files:
                    # '의 사본', '복사본', '의 사본 (1)' 등 모든 케이스 대응 및 Jamo 형태의 '의 사본' 추가
                    targets_to_clean = ["의 사본", "사본", "복사본", "Copy", "의 사본"] # Jamo 형태 추가

                    if any(target in file for target in targets_to_clean):
                        # print(f"\n--- 파일명 정화 시도: {f_name}/{sub} ---") # 상세 로그 제거
                        # print(f"  원본 파일: {file}") # 상세 로그 제거
                        old_path = os.path.join(sub_path, file)

                        ext = ""
                        if ".jpg" in file.lower(): ext = ".jpg"
                        elif ".png" in file.lower(): ext = ".png"
                        elif ".jpeg" in file.lower(): ext = ".jpeg"

                        if ext:
                            # 확장자 앞부분만 추출 (예: xxxxx.jpg 까지만 남김)
                            clean_name = file.split(ext)[0] + ext
                            new_path = os.path.join(sub_path, clean_name)

                            # print(f"  예상 새 파일명: {clean_name}") # 상세 로그 제거
                            # print(f"  예상 새 경로: {new_path}") # 상세 로그 제거

                            try:
                                if not os.path.exists(new_path):
                                    os.rename(old_path, new_path)
                                    fixed_count += 1
                                    # print(f"  ✅ '{file}' -> '{clean_name}' (이름 변경 완료)") # 상세 로그 제거
                                else:
                                    # 만약 이미 깨끗한 파일이 있다면 사본은 그냥 삭제
                                    os.remove(old_path)
                                    fixed_count += 1
                                    # print(f"  🗑️ '{file}' (중복된 깨끗한 파일이 있어 삭제)") # 상세 로그 제거
                            except Exception as e:
                                print(f"  ⚠️ '{file}' 변경 중 오류: {e}") # 오류 로그는 유지
                        else:
                            print(f"  ❌ '{file}'은/는 지원되는 이미지 확장자를 찾을 수 없어 정화 대상에서 제외됩니다.") # 로그 유지
                # ------------------------------

                # 정화 후 파일 목록 다시 읽기
                final_files = [i for i in os.listdir(sub_path) if i.lower().endswith(('.jpg', '.jpeg', '.png'))]
                cnt = len(final_files)

                icon = "🟢" if cnt == 8 else "🔴"
                if cnt != 8: is_all_8 = False
                status_list.append(f"{icon} {sub}({cnt})")
            else:
                status_list.append(f"⚪ {sub}(미생성)")
                is_all_8 = False

        row_content = f"| {'✅' if is_all_8 else '❌'} | `{f_name}` | {' / '.join(status_list)} |"

        if not is_all_8:
            error_count += 1
            problem_reports.append(row_content) # 문제가 있는 폴더만 저장

    if fixed_count > 0:
        print(f"\n🛠  파일명 수정/정리 완료: 총 {fixed_count}건")

    normal_folders_count = total_folders_count - error_count

    # 전체 요약 보고서
    summary_markdown = f"""
### 📊 검증 보고서 (`{카테고리}`)
* 총 폴더 개수: {total_folders_count}개
* 정상 폴더 개수: {normal_folders_count}개
* 문제 발견 폴더: {error_count}개
"""
    display(Markdown(summary_markdown))

    if error_count == 0:
        display(Markdown("✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다."))
    else:
        # 문제 폴더를 모두 출력 (display_limit 제거)
        problem_report_markdown = f"""
---
#### ❌ 문제가 있는 폴더 상세 보고 (총 {error_count}개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
{       '\n' .join(problem_reports)}
"""
        display(Markdown(problem_report_markdown))

run_verification()


### 📊 검증 보고서 (`vandalism`)
* 총 폴더 개수: 606개
* 정상 폴더 개수: 210개
* 문제 발견 폴더: 396개



---
#### ❌ 문제가 있는 폴더 상세 보고 (총 396개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
| ❌ | `(33kib)706-1_cam01_vandalism02_place01_day_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `(33kib)706-1_cam01_vandalism02_place01_day_spring (1)` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `(미완)87-5_cam01_vandalism01_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-2_cam01_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-2_cam01_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-2_cam02_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-2_cam02_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-3_cam01_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-3_cam01_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-3_cam02_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-3_cam02_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-4_cam01_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-4_cam01_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-4_cam02_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-4_cam02_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-5_cam01_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-5_cam01_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-5_cam02_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-5_cam02_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-6_cam01_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-6_cam01_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-6_cam02_vandalism02_place02_night_spring` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `83-6_cam02_vandalism02_place02_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `84-1_cam01_vandalism01_place09_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `84-1_cam02_vandalism01_place09_night_summer` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_13_BU_SMA_09-07_15-31-29_CC_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_13_BU_SMA_09-07_15-31-29_CD_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_23_BU_SYB_10-04_10-49-52_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMA_09-27_11-33-56_CA_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMA_09-27_11-33-56_CB_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMA_09-27_11-33-56_CC_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMA_09-27_11-33-56_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMB_09-02_15-39-44_CA_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMB_09-02_15-39-44_CC_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SMB_09-02_15-39-44_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYA_10-06_12-33-09_CB_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYA_10-06_12-33-09_CC_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYA_10-06_12-33-09_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYB_10-04_10-47-42_CA_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYB_10-04_10-47-42_CB_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYB_10-04_10-47-42_CC_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_24_BU_SYB_10-04_10-47-42_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMA_09-27_10-37-17_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMA_09-27_10-37-17_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMA_09-27_10-37-17_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMA_09-27_10-37-17_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMB_09-02_13-41-37_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMB_09-02_13-41-37_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SMB_09-02_13-41-37_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYA_10-06_12-35-06_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYA_10-06_12-35-06_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYA_10-06_12-35-06_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYA_10-06_12-35-06_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYB_10-04_11-41-08_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYB_10-04_11-41-08_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYB_10-04_11-41-08_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_25_BU_SYB_10-04_11-41-08_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SMA_09-27_10-39-31_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SMA_09-27_10-39-31_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SMA_09-27_10-39-31_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SMA_09-27_10-39-31_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYA_10-06_12-37-03_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYA_10-06_12-37-03_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYA_10-06_12-37-03_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYA_10-06_12-37-03_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYB_10-04_11-43-07_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYB_10-04_11-43-07_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_26_BU_SYB_10-04_11-43-07_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_27_BU_DYA_08-23_11-50-31_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_27_BU_DYA_08-23_11-50-33_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_27_BU_DYA_08-23_11-50-33_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SMA_09-27_10-44-22_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SMA_09-27_10-44-22_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SMA_09-27_10-44-22_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SMA_09-27_10-44-22_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYA_10-06_12-38-51_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYA_10-06_12-38-51_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYA_10-06_12-38-51_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYA_10-06_12-38-51_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYB_10-04_11-45-02_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYB_10-04_11-45-02_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYB_10-04_11-45-02_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_28_BU_SYB_10-04_11-45-02_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMA_09-27_10-48-06_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMA_09-27_10-48-06_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMA_09-27_10-48-06_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMA_09-27_10-48-06_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMB_09-02_13-53-36_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMB_09-02_13-53-36_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMB_09-02_13-53-36_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SMC_10-16_10-30-38_CC_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYA_10-06_12-40-22_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYA_10-06_12-40-22_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYA_10-06_12-40-22_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYA_10-06_12-40-22_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYB_10-04_11-51-32_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_29_BU_SYB_10-04_11-51-32_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_30_BU_DYA_08-23_11-02-08_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_30_BU_DYA_08-23_11-02-10_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_30_BU_SMA_09-27_10-49-40_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_30_BU_SMA_09-27_10-49-40_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_30_BU_SMA_09-27_10-49-40_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_53_BU_DYB_10-17_11-14-01_CA_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_57_BU_DYA_08-23_15-06-11_CD_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_57_BU_DYA_08-23_15-06-11_CE_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_57_BU_DYA_08-23_15-06-11_CF_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_57_BU_SMC_10-14_13-40-47_CE_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_59_BU_DYB_10-17_13-22-05_CE_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_59_BU_SMC_10-13_16-36-18_CC_RGB_DF2_M4` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_59_BU_SMC_10-13_16-36-18_CD_RGB_DF2_M4` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_62_BU_SMC_10-13_16-43-09_CE_RGB_DF2_M4` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_63_BU_DYB_10-17_13-29-39_CA_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_63_BU_DYB_10-17_13-29-39_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_8_6_BU_SMA_08-28_13-46-05_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-00-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-07-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-07-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-07-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-11-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-11-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-12-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-21-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-21-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-21-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-25-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-25-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-26-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-30-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-30-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-30-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-34-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-34-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-34-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-38-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-38-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-38-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-43-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-43-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-43-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-47-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-47-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-47-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-51-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-51-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-51-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-55-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-55-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-55-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-59-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-59-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_1_smp_cl_08-29_18-59-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-46-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-46-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-46-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-56-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-56-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_10-57-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-27-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-27-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-27-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-31-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-32-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-32-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-36-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-36-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-36-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-41-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-41-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-41-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-50-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-50-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-51-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-55-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-55-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_11-55-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-00-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-00-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-00-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-05-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-05-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-05-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-10-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-10-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-10-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-33-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-33-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_12-33-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-48-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-48-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-48-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-53-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-53-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-53-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-57-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-57-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_13-57-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-08-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-08-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-08-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-13-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-13-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_cl_08-29_14-14-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-15-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-15-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-15-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-19-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-19-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-19-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-24-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-24-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-25-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-29-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-29-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_2_smp_ra_08-29_12-29-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-41-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-41-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-41-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-45-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-45-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-45-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-49-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-49-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-49-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-54-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-54-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_10-54-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-08-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-08-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-09-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-14-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-14-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-14-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-18-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-18-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-18-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-23-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-23-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-23-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-28-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-28-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-28-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-32-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-32-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-32-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-36-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-36-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-36-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-40-00_a_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-40-00_b_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_11-40-00_c_for_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-08-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-08-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-08-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-13-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-13-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-13-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-17-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-17-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-17-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-21-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-21-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-21-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-35-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-35-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-35-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-40-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-40-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-40-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-44-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-44-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-44-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-48-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-48-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-48-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-52-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-52-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-52-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-56-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-56-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_13-57-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-01-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-01-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-01-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-05-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-05-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-05-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-23-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-23-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-23-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-28-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-28-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-28-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-32-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-32-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-32-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-36-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-36-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-37-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-42-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-42-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-43-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-47-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-47-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_14-47-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-03-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-03-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-03-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-08-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-08-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-09-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-15-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-15-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-15-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-19-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-19-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-19-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-24-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-24-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-24-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-28-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-28-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_3_smp_su_08-30_15-28-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-41-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-41-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-41-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-45-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-45-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-45-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-49-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-49-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-49-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-53-00_a_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-53-00_b_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_4_ggjp_su_08-30_18-53-00_c_set_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-28-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-28-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-32-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-32-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-32-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-35-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-35-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-36-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-39-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-39-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-40-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-43-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-43-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_5_smp_cl_09-10_16-44-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-17-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-17-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-22-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-22-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-22-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-26-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-26-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-26-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-30-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-30-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-30-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-34-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-34-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_6_smp_su_08-30_16-34-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_16-51-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_16-51-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_16-56-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_16-56-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_16-56-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-08-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-08-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-08-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-13-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-13-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-13-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-17-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-17-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-17-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-22-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-22-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-22-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-26-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-26-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_7_smp_su_08-30_17-27-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-49-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-49-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-53-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-53-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-53-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-57-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-57-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_17-57-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-01-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-01-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-01-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-06-00_a_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-06-00_b_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_5_8_smp_su_08-30_18-06-00_c_aft_DF1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |


In [8]:
"""
투기(Dump)
"""
def run_verification():
    if not os.path.exists(base_path):
        print(f"❌ 경로를 찾을 수 없습니다: {base_path}")
        return

    exclude_list = ["전체", "ipynb_checkpoints", ".ipynb_checkpoints"]
    folders = sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f not in exclude_list])

    problem_reports = [] # 문제가 있는 폴더만 저장
    error_count = 0
    fixed_count = 0
    total_folders_count = len(folders)

    for f_name in folders:
        f_path = os.path.join(base_path, f_name)
        status_list, is_all_8 = [], True

        for sub in ["이상", "의심", "정상"]:
            sub_path = os.path.join(f_path, sub)
            if os.path.exists(sub_path):

                # --- [파일명 정화 로직 강화] ---
                current_files = os.listdir(sub_path)
                for file in current_files:
                    # '의 사본', '복사본', '의 사본 (1)' 등 모든 케이스 대응 및 Jamo 형태의 '의 사본' 추가
                    targets_to_clean = ["의 사본", "사본", "복사본", "Copy", "의 사본"] # Jamo 형태 추가

                    if any(target in file for target in targets_to_clean):
                        # print(f"\n--- 파일명 정화 시도: {f_name}/{sub} ---") # 상세 로그 제거
                        # print(f"  원본 파일: {file}") # 상세 로그 제거
                        old_path = os.path.join(sub_path, file)

                        ext = ""
                        if ".jpg" in file.lower(): ext = ".jpg"
                        elif ".png" in file.lower(): ext = ".png"
                        elif ".jpeg" in file.lower(): ext = ".jpeg"

                        if ext:
                            # 확장자 앞부분만 추출 (예: xxxxx.jpg 까지만 남김)
                            clean_name = file.split(ext)[0] + ext
                            new_path = os.path.join(sub_path, clean_name)

                            # print(f"  예상 새 파일명: {clean_name}") # 상세 로그 제거
                            # print(f"  예상 새 경로: {new_path}") # 상세 로그 제거

                            try:
                                if not os.path.exists(new_path):
                                    os.rename(old_path, new_path)
                                    fixed_count += 1
                                    # print(f"  ✅ '{file}' -> '{clean_name}' (이름 변경 완료)") # 상세 로그 제거
                                else:
                                    # 만약 이미 깨끗한 파일이 있다면 사본은 그냥 삭제
                                    os.remove(old_path)
                                    fixed_count += 1
                                    # print(f"  🗑️ '{file}' (중복된 깨끗한 파일이 있어 삭제)") # 상세 로그 제거
                            except Exception as e:
                                print(f"  ⚠️ '{file}' 변경 중 오류: {e}") # 오류 로그는 유지
                        else:
                            print(f"  ❌ '{file}'은/는 지원되는 이미지 확장자를 찾을 수 없어 정화 대상에서 제외됩니다.") # 로그 유지
                # ------------------------------

                # 정화 후 파일 목록 다시 읽기
                final_files = [i for i in os.listdir(sub_path) if i.lower().endswith(('.jpg', '.jpeg', '.png'))]
                cnt = len(final_files)

                icon = "🟢" if cnt == 8 else "🔴"
                if cnt != 8: is_all_8 = False
                status_list.append(f"{icon} {sub}({cnt})")
            else:
                status_list.append(f"⚪ {sub}(미생성)")
                is_all_8 = False

        row_content = f"| {'✅' if is_all_8 else '❌'} | `{f_name}` | {' / '.join(status_list)} |"

        if not is_all_8:
            error_count += 1
            problem_reports.append(row_content) # 문제가 있는 폴더만 저장

    if fixed_count > 0:
        print(f"\n🛠  파일명 수정/정리 완료: 총 {fixed_count}건")

    normal_folders_count = total_folders_count - error_count

    # 전체 요약 보고서
    summary_markdown = f"""
### 📊 검증 보고서 (`{카테고리}`)
* 총 폴더 개수: {total_folders_count}개
* 정상 폴더 개수: {normal_folders_count}개
* 문제 발견 폴더: {error_count}개
"""
    display(Markdown(summary_markdown))

    if error_count == 0:
        display(Markdown("✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다."))
    else:
        # 문제 폴더를 모두 출력 (display_limit 제거)
        problem_report_markdown = f"""
---
#### ❌ 문제가 있는 폴더 상세 보고 (총 {error_count}개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
{       '\n' .join(problem_reports)}
"""
        display(Markdown(problem_report_markdown))

run_verification()


### 📊 검증 보고서 (`dump`)
* 총 폴더 개수: 210개
* 정상 폴더 개수: 210개
* 문제 발견 폴더: 0개


✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다.

In [10]:
"""
폭행(Assault)
"""
def run_verification():
    if not os.path.exists(base_path):
        print(f"❌ 경로를 찾을 수 없습니다: {base_path}")
        return

    exclude_list = ["전체", "ipynb_checkpoints", ".ipynb_checkpoints"]
    folders = sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f not in exclude_list])

    problem_reports = [] # 문제가 있는 폴더만 저장
    error_count = 0
    fixed_count = 0
    total_folders_count = len(folders)

    for f_name in folders:
        f_path = os.path.join(base_path, f_name)
        status_list, is_all_8 = [], True

        for sub in ["이상", "의심", "정상"]:
            sub_path = os.path.join(f_path, sub)
            if os.path.exists(sub_path):

                # --- [파일명 정화 로직 강화] ---
                current_files = os.listdir(sub_path)
                for file in current_files:
                    # '의 사본', '복사본', '의 사본 (1)' 등 모든 케이스 대응 및 Jamo 형태의 '의 사본' 추가
                    targets_to_clean = ["의 사본", "사본", "복사본", "Copy", "의 사본"] # Jamo 형태 추가

                    if any(target in file for target in targets_to_clean):
                        # print(f"\n--- 파일명 정화 시도: {f_name}/{sub} ---") # 상세 로그 제거
                        # print(f"  원본 파일: {file}") # 상세 로그 제거
                        old_path = os.path.join(sub_path, file)

                        ext = ""
                        if ".jpg" in file.lower(): ext = ".jpg"
                        elif ".png" in file.lower(): ext = ".png"
                        elif ".jpeg" in file.lower(): ext = ".jpeg"

                        if ext:
                            # 확장자 앞부분만 추출 (예: xxxxx.jpg 까지만 남김)
                            clean_name = file.split(ext)[0] + ext
                            new_path = os.path.join(sub_path, clean_name)

                            # print(f"  예상 새 파일명: {clean_name}") # 상세 로그 제거
                            # print(f"  예상 새 경로: {new_path}") # 상세 로그 제거

                            try:
                                if not os.path.exists(new_path):
                                    os.rename(old_path, new_path)
                                    fixed_count += 1
                                    # print(f"  ✅ '{file}' -> '{clean_name}' (이름 변경 완료)") # 상세 로그 제거
                                else:
                                    # 만약 이미 깨끗한 파일이 있다면 사본은 그냥 삭제
                                    os.remove(old_path)
                                    fixed_count += 1
                                    # print(f"  🗑️ '{file}' (중복된 깨끗한 파일이 있어 삭제)") # 상세 로그 제거
                            except Exception as e:
                                print(f"  ⚠️ '{file}' 변경 중 오류: {e}") # 오류 로그는 유지
                        else:
                            print(f"  ❌ '{file}'은/는 지원되는 이미지 확장자를 찾을 수 없어 정화 대상에서 제외됩니다.") # 로그 유지
                # ------------------------------

                # 정화 후 파일 목록 다시 읽기
                final_files = [i for i in os.listdir(sub_path) if i.lower().endswith(('.jpg', '.jpeg', '.png'))]
                cnt = len(final_files)

                icon = "🟢" if cnt == 8 else "🔴"
                if cnt != 8: is_all_8 = False
                status_list.append(f"{icon} {sub}({cnt})")
            else:
                status_list.append(f"⚪ {sub}(미생성)")
                is_all_8 = False

        row_content = f"| {'✅' if is_all_8 else '❌'} | `{f_name}` | {' / '.join(status_list)} |"

        if not is_all_8:
            error_count += 1
            problem_reports.append(row_content) # 문제가 있는 폴더만 저장

    if fixed_count > 0:
        print(f"\n🛠  파일명 수정/정리 완료: 총 {fixed_count}건")

    normal_folders_count = total_folders_count - error_count

    # 전체 요약 보고서
    summary_markdown = f"""
### 📊 검증 보고서 (`{카테고리}`)
* 총 폴더 개수: {total_folders_count}개
* 정상 폴더 개수: {normal_folders_count}개
* 문제 발견 폴더: {error_count}개
"""
    display(Markdown(summary_markdown))

    if error_count == 0:
        display(Markdown("✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다."))
    else:
        # 문제 폴더를 모두 출력 (display_limit 제거)
        problem_report_markdown = f"""
---
#### ❌ 문제가 있는 폴더 상세 보고 (총 {error_count}개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
{       '\n' .join(problem_reports)}
"""
        display(Markdown(problem_report_markdown))

run_verification()


🛠  파일명 수정/정리 완료: 총 1건



### 📊 검증 보고서 (`assault`)
* 총 폴더 개수: 210개
* 정상 폴더 개수: 210개
* 문제 발견 폴더: 0개


✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다.

In [19]:
"""
절도(Burglary)
"""
def run_verification():
    if not os.path.exists(base_path):
        print(f"❌ 경로를 찾을 수 없습니다: {base_path}")
        return

    exclude_list = ["전체", "ipynb_checkpoints", ".ipynb_checkpoints"]
    folders = sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f not in exclude_list])

    problem_reports = [] # 문제가 있는 폴더만 저장
    error_count = 0
    fixed_count = 0
    total_folders_count = len(folders)

    for f_name in folders:
        f_path = os.path.join(base_path, f_name)
        status_list, is_all_8 = [], True

        for sub in ["이상", "의심", "정상"]:
            sub_path = os.path.join(f_path, sub)
            if os.path.exists(sub_path):

                # --- [파일명 정화 로직 강화] ---
                current_files = os.listdir(sub_path)
                for file in current_files:
                    # '의 사본', '복사본', '의 사본 (1)' 등 모든 케이스 대응 및 Jamo 형태의 '의 사본' 추가
                    targets_to_clean = ["의 사본", "사본", "복사본", "Copy", "의 사본"] # Jamo 형태 추가

                    if any(target in file for target in targets_to_clean):
                        # print(f"\n--- 파일명 정화 시도: {f_name}/{sub} ---") # 상세 로그 제거
                        # print(f"  원본 파일: {file}") # 상세 로그 제거
                        old_path = os.path.join(sub_path, file)

                        ext = ""
                        if ".jpg" in file.lower(): ext = ".jpg"
                        elif ".png" in file.lower(): ext = ".png"
                        elif ".jpeg" in file.lower(): ext = ".jpeg"

                        if ext:
                            # 확장자 앞부분만 추출 (예: xxxxx.jpg 까지만 남김)
                            clean_name = file.split(ext)[0] + ext
                            new_path = os.path.join(sub_path, clean_name)

                            # print(f"  예상 새 파일명: {clean_name}") # 상세 로그 제거
                            # print(f"  예상 새 경로: {new_path}") # 상세 로그 제거

                            try:
                                if not os.path.exists(new_path):
                                    os.rename(old_path, new_path)
                                    fixed_count += 1
                                    # print(f"  ✅ '{file}' -> '{clean_name}' (이름 변경 완료)") # 상세 로그 제거
                                else:
                                    # 만약 이미 깨끗한 파일이 있다면 사본은 그냥 삭제
                                    os.remove(old_path)
                                    fixed_count += 1
                                    # print(f"  🗑️ '{file}' (중복된 깨끗한 파일이 있어 삭제)") # 상세 로그 제거
                            except Exception as e:
                                print(f"  ⚠️ '{file}' 변경 중 오류: {e}") # 오류 로그는 유지
                        else:
                            print(f"  ❌ '{file}'은/는 지원되는 이미지 확장자를 찾을 수 없어 정화 대상에서 제외됩니다.") # 로그 유지
                # ------------------------------

                # 정화 후 파일 목록 다시 읽기
                final_files = [i for i in os.listdir(sub_path) if i.lower().endswith(('.jpg', '.jpeg', '.png'))]
                cnt = len(final_files)

                icon = "🟢" if cnt == 8 else "🔴"
                if cnt != 8: is_all_8 = False
                status_list.append(f"{icon} {sub}({cnt})")
            else:
                status_list.append(f"⚪ {sub}(미생성)")
                is_all_8 = False

        row_content = f"| {'✅' if is_all_8 else '❌'} | `{f_name}` | {' / '.join(status_list)} |"

        if not is_all_8:
            error_count += 1
            problem_reports.append(row_content) # 문제가 있는 폴더만 저장

    if fixed_count > 0:
        print(f"\n🛠  파일명 수정/정리 완료: 총 {fixed_count}건")

    normal_folders_count = total_folders_count - error_count

    # 전체 요약 보고서
    summary_markdown = f"""
### 📊 검증 보고서 (`{카테고리}`)
* 총 폴더 개수: {total_folders_count}개
* 정상 폴더 개수: {normal_folders_count}개
* 문제 발견 폴더: {error_count}개
"""
    display(Markdown(summary_markdown))

    if error_count == 0:
        display(Markdown("✅ 모든 폴더가 정상적으로 검증되었습니다. 문제가 있는 폴더가 없습니다."))
    else:
        # 문제 폴더를 모두 출력 (display_limit 제거)
        problem_report_markdown = f"""
---
#### ❌ 문제가 있는 폴더 상세 보고 (총 {error_count}개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
{       '\n' .join(problem_reports)}
"""
        display(Markdown(problem_report_markdown))

run_verification()


### 📊 검증 보고서 (`burglary`)
* 총 폴더 개수: 278개
* 정상 폴더 개수: 210개
* 문제 발견 폴더: 68개



---
#### ❌ 문제가 있는 폴더 상세 보고 (총 68개)

| 상태 | 폴더명 | 상세 정보 |
|:---:|:---|:---|
| ❌ | `C_3_12_24_BU_SYB_10-04_14-50-14_CC_RGB_DF2_M3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_25_BU_DYA_08-12_13-48-02_CC_RGB_DF2_M4` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_25_BU_DYB_08-06_16-42-03_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_25_BU_SYA_10-06_14-46-19_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_25_BU_SYA_10-06_14-46-19_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_25_BU_SYB_10-04_14-51-46_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_26_BU_SYA_10-06_14-47-44_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_26_BU_SYA_10-06_14-47-44_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_26_BU_SYB_10-04_14-53-12_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_27_BU_DYB_08-06_16-48-45_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_27_BU_SYA_10-06_14-49-27_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_27_BU_SYB_10-04_14-54-48_CB_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_27_BU_SYB_10-04_14-54-48_CC_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_28_BU_DYB_08-06_16-51-09_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_29_BU_DYA_07-31_14-14-15_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_29_BU_DYA_07-31_14-14-17_CE_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_29_BU_DYA_08-10_16-50-14_CA_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_29_BU_DYA_08-10_16-50-20_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_DYA_07-31_14-21-46_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_DYA_07-31_14-21-49_CE_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_DYA_07-31_14-21-49_CF_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_DYA_08-10_16-51-56_CA_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_DYA_08-10_16-52-02_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_30_BU_SYA_10-06_14-37-31_CD_RGB_DF2_F3` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_DYA_07-31_14-23-44_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_DYA_07-31_14-23-47_CE_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_DYA_07-31_14-23-47_CF_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_DYA_08-10_16-53-54_CA_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_DYA_08-10_16-54-00_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_31_BU_SMC_10-16_11-00-59_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_DYA_07-31_14-26-24_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_DYA_07-31_14-26-27_CE_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_DYA_07-31_14-26-27_CF_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_DYA_08-10_16-56-34_CA_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_DYA_08-10_16-56-40_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_SMC_10-16_11-02-57_CA_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_SMC_10-16_11-02-57_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_32_BU_SMC_10-16_11-02-57_CE_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_33_BU_DYA_07-29_11-52-59_CD_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_33_BU_DYA_07-29_11-52-59_CE_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_33_BU_DYA_08-10_16-58-28_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_34_BU_DYA_07-29_12-00-36_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_34_BU_DYA_07-29_12-00-36_CE_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_34_BU_DYA_08-10_17-14-02_CA_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_34_BU_DYA_08-10_17-14-10_CC_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_34_BU_SMB_09-05_13-56-05_CD_RGB_DF2_F4` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_DYA_07-29_12-02-40_CD_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_DYA_07-29_12-02-40_CE_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_DYA_08-10_17-17-30_CA_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_DYA_08-10_17-17-36_CC_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_SMC_10-16_11-11-42_CC_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_35_BU_SMC_10-16_11-11-42_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_DYA_07-29_12-05-40_CD_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_DYA_07-29_12-05-40_CE_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_DYA_07-29_12-05-40_CF_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_DYA_08-10_17-19-23_CA_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_DYA_08-10_17-19-28_CC_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_SMC_10-16_11-16-23_CA_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_SMC_10-16_11-16-23_CC_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_36_BU_SMC_10-16_11-16-23_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_37_BU_DYA_07-29_12-07-45_CD_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_37_BU_DYA_07-29_12-07-45_CE_RGB_DF2_M2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_37_BU_DYA_08-10_17-21-18_CA_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_37_BU_DYA_08-10_17-21-24_CC_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_38_BU_DYA_08-10_17-23-04_CA_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_38_BU_DYA_08-10_17-23-10_CC_RGB_DF2_F2` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_38_BU_SMC_10-16_11-18-38_CD_RGB_DF2_F1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
| ❌ | `C_3_12_39_BU_DYB_10-16_14-45-25_CD_RGB_DF2_M1` | 🔴 이상(0) / 🔴 의심(0) / 🔴 정상(0) |
